# (Dynamic) Multiple Dispatch in Julia

[Multiple dispatch](http://en.wikipedia.org/wiki/Multiple_dispatch) can be thought of as a generalization of **object-oriented** (OO) programming.

In a typical OO language like Python, an object type (class) *owns* certain *methods* (functions), and are typically called via
```
object.method(arg1, arg2)
```
Depending on the type of `object`, the runtime system will *dispatch* to different `method` definitions.

In Julia, the same call would be "spelled" differently:
```
method(object, arg1, arg2)
```
Spelled this way, you should notice something odd about OO programming: why is the *first* argument so special?

Traditional OO programming corresponds to **single dispatch**: the runtime chooses `method` based on the type of the *first* argument only.   Julia implements **multiple dispatch**: the runtime chooses `method` based on the types of *all* the arguments.

## Example: Binary mathematical operators

A classic example of the need for multiple dispatch is the case of binary math operators.  If you compute `x * y`, the definition of the `*` function depends upon *both* the arguments, not just on `x`.

Julia defines *many versions* of the `*` function:

In [1]:
methods(*)

# 138 methods for generic function "*":
*(x::Bool, y::Bool) at bool.jl:38
*{T<:Unsigned}(x::Bool, y::T<:Unsigned) at bool.jl:53
*(x::Bool, z::Complex{Bool}) at complex.jl:122
*(x::Bool, z::Complex{T<:Real}) at complex.jl:129
*{T<:Number}(x::Bool, y::T<:Number) at bool.jl:49
*(x::Float32, y::Float32) at float.jl:211
*(x::Float64, y::Float64) at float.jl:212
*(z::Complex{T<:Real}, w::Complex{T<:Real}) at complex.jl:113
*(z::Complex{Bool}, x::Bool) at complex.jl:123
*(z::Complex{T<:Real}, x::Bool) at complex.jl:130
*(x::Real, z::Complex{Bool}) at complex.jl:140
*(z::Complex{Bool}, x::Real) at complex.jl:141
*(x::Real, z::Complex{T<:Real}) at complex.jl:152
*(z::Complex{T<:Real}, x::Real) at complex.jl:153
*(x::Rational{T<:Integer}, y::Rational{T<:Integer}) at rational.jl:186
*(a::Float16, b::Float16) at float16.jl:136
*{N}(a::Integer, index::CartesianIndex{N}) at multidimensional.jl:50
*(x::BigInt, y::BigInt) at gmp.jl:256
*(a::BigInt, b::BigInt, c::BigInt) at gmp.jl:279
*(a::BigInt, b::BigInt, c::BigInt, d::BigInt) at gmp.jl:285
*(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) at gmp.jl:292
*(x::BigInt, c::Union{UInt16,UInt32,UInt64,UInt8}) at gmp.jl:326
*(c::Union{UInt16,UInt32,UInt64,UInt8}, x::BigInt) at gmp.jl:330
*(x::BigInt, c::Union{Int16,Int32,Int64,Int8}) at gmp.jl:332
*(c::Union{Int16,Int32,Int64,Int8}, x::BigInt) at gmp.jl:336
*(x::BigFloat, y::BigFloat) at mpfr.jl:208
*(x::BigFloat, c::Union{UInt16,UInt32,UInt64,UInt8}) at mpfr.jl:215
*(c::Union{UInt16,UInt32,UInt64,UInt8}, x::BigFloat) at mpfr.jl:219
*(x::BigFloat, c::Union{Int16,Int32,Int64,Int8}) at mpfr.jl:223
*(c::Union{Int16,Int32,Int64,Int8}, x::BigFloat) at mpfr.jl:227
*(x::BigFloat, c::Union{Float16,Float32,Float64}) at mpfr.jl:231
*(c::Union{Float16,Float32,Float64}, x::BigFloat) at mpfr.jl:235
*(x::BigFloat, c::BigInt) at mpfr.jl:239
*(c::BigInt, x::BigFloat) at mpfr.jl:243
*(a::BigFloat, b::BigFloat, c::BigFloat) at mpfr.jl:379
*(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) at mpfr.jl:385
*(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) at mpfr.jl:392
*{T<:Number}(x::T<:Number, D::Diagonal{T}) at linalg/diagonal.jl:89
*(x::Irrational{sym}, y::Irrational{sym}) at irrationals.jl:72
*(y::Real, x::Base.Dates.Period) at dates/periods.jl:74
*(x::Number) at operators.jl:74
*(y::Number, x::Bool) at bool.jl:55
*(x::Int8, y::Int8) at int.jl:19
*(x::UInt8, y::UInt8) at int.jl:19
*(x::Int16, y::Int16) at int.jl:19
*(x::UInt16, y::UInt16) at int.jl:19
*(x::Int32, y::Int32) at int.jl:19
*(x::UInt32, y::UInt32) at int.jl:19
*(x::Int64, y::Int64) at int.jl:19
*(x::UInt64, y::UInt64) at int.jl:19
*(x::Int128, y::Int128) at int.jl:456
*(x::UInt128, y::UInt128) at int.jl:457
*{T<:Number}(x::T<:Number, y::T<:Number) at promotion.jl:212
*(x::Number, y::Number) at promotion.jl:168
*{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},S}(A::Union{DenseArray{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},2},SubArray{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},2,A<:DenseArray{T,N},I<:Tuple{Vararg{Union{Colon,Int64,Range{Int64}}}},LD}}, x::Union{DenseArray{S,1},SubArray{S,1,A<:DenseArray{T,N},I<:Tuple{Vararg{Union{Colon,Int64,Range{Int64}}}},LD}}) at linalg/matmul.jl:82
*(A::SymTridiagonal{T}, B::Number) at linalg/tridiag.jl:86
*(A::Tridiagonal{T}, B::Number) at linalg/tridiag.jl:406
*(A::UpperTriangular{T,S<:AbstractArray{T,2}}, x::Number) at linalg/triangular.jl:454
*(A::Base.LinAlg.UnitUpperTriangular{T,S<:AbstractArray{T,2}}, x::Number) at linalg/triangular.jl:457
*(A::LowerTriangular{T,S<:AbstractArray{T,2}}, x::Number) at linalg/triangular.jl:454
*(A::Base.LinAlg.UnitLowerTriangular{T,S<:AbstractArray{T,2}}, x::Number) at linalg/triangular.jl:457
*(A::Tridiagonal{T}, B::UpperTriangular{T,S<:AbstractArray{T,2}}) at linalg/triangular.jl:969
*(A::Tridiagonal{T}, B::Base.LinAlg.UnitUpperTriangular{T,S<:AbstractArray{T,2}}) at linalg/triangular.jl:969
*(A::Tridiagonal{T}, B::LowerTriangular{T,S<:AbstractArray{T

We can add new methods to a given function at any time.  The methods don't "belong" to a particular type, and aren't part of the type's definition.

For example, string concatenation in Julia is done via `*`:

In [2]:
"hello" * "world"

"helloworld"

In [3]:
"hello" + "world"

LoadError: LoadError: MethodError: `+` has no method matching +(::ASCIIString, ::ASCIIString)
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...)
while loading In[3], in expression starting on line 1

But we can easily extend `+` to support a concatenation for strings, if we want:

In [4]:
import Base.+ # we must import a method to add methods (as opposed to replacing it)
+(x::String, y::String) = x * " " * y

  likely near In[4]:2
  likely near In[4]:2


+ (generic function with 172 methods)

In [5]:
"hello" + "world"

"hello world"

## Not the same as C++ overloading: Dynamic vs. static

This may look a lot like function overloading in languages like C++.   The difference is that C++'s overloading is **static** (= *dispatch at compile-time*), whereas Julia's overloading is dynamic (= *dispatch at run-time*), like OO polymorphism.

For example, now that we've defined `+`, we can use strings with any previously defined function that requires a `+` operation, like `sum` (summation):

In [6]:
sum(["The", "quick", "brown", "fox", "jumped", "over", "the", "lazy", "dog."])

"The quick brown fox jumped over the lazy dog."

## Type declarations are "function filters"

Type declarations are *not* required for performance — Julia automatically specializes a function on its argument types during compilation.  They act like **filters**, allowing us to specify *which functions are used when*.

Without this, in a language like Python, you sometimes have to write *manual function filters* like this example from Matplotlib's [quiver.py](https://github.com/matplotlib/matplotlib/blob/43150ef012e405de5c2fb7b779d1bffa77f7af81/lib/matplotlib/quiver.py):
```python
def _parse_args(*args):
    X, Y, U, V, C = [None] * 5
    args = list(args)
    # The use of atleast_1d allows for handling scalar arguments while also
    # keeping masked arrays
    if len(args) == 3 or len(args) == 5:
        C = np.atleast_1d(args.pop(-1))
    V = np.atleast_1d(args.pop(-1))
    U = np.atleast_1d(args.pop(-1))
    if U.ndim == 1:
        nr, nc = 1, U.shape[0]
    else:
        nr, nc = U.shape
    if len(args) == 2: # remaining after removing U,V,C
        X, Y = [np.array(a).ravel() for a in args]
        if len(X) == nc and len(Y) == nr:
            X, Y = [a.ravel() for a in np.meshgrid(X, Y)]
    else:
        indexgrid = np.meshgrid(np.arange(nc), np.arange(nr))
        X, Y = [np.ravel(a) for a in indexgrid]
    return X, Y, U, V, C
```
In Julia, you could define different methods for differing numbers of arguments, arrays vs. scalars, etcetera (all eventually calling a single lower-level function to do the work once the arguments have been transformed).